# 06 angr symbolic dynamic memory

Let's analyze this assembly code:
```
                             **************************************************************
                             *                          FUNCTION                          *
                             **************************************************************
                             int __cdecl main(int argc, char * * argv)
        001008bc ff 83 02 d1     sub        sp,sp,#0xa0
        001008c0 fd 7b 09 a9     stp        x29,x30,[sp, #local_10]
        001008c4 fd 43 02 91     add        x29,sp,#0x90
        001008c8 bf c3 1b b8     stur       wzr,[x29, #local_54]
        001008cc e0 4b 00 b9     str        argc,[sp, #local_58]
        001008d0 e1 23 00 f9     str        argv,[sp, #local_60]
        001008d4 20 01 80 d2     mov        argc,#0x9
        001008d8 e0 13 00 f9     str        argc,[sp, #local_80]
        001008dc 89 00 00 94     bl         <EXTERNAL>::malloc                               void * malloc(size_t __size)
        001008e0 e8 03 00 aa     mov        x8,argc
        001008e4 e0 13 40 f9     ldr        argc,[sp, #local_80]
        001008e8 09 00 00 d0     adrp       x9,0x102000
        001008ec e9 17 00 f9     str        x9,[sp, #local_78]
        001008f0 28 f5 06 f9     str        x8,[x9, #0xde8]=>buffer0                         = NaP
        001008f4 83 00 00 94     bl         <EXTERNAL>::malloc                               void * malloc(size_t __size)
        001008f8 e8 13 40 f9     ldr        x8,[sp, #local_80]
        001008fc e9 17 40 f9     ldr        x9,[sp, #local_78]
        00100900 0a 00 00 d0     adrp       x10,0x102000
        00100904 ea 1b 00 f9     str        x10,[sp, #local_70]
        00100908 40 f9 06 f9     str        argc,[x10, #0xdf0]=>buffer1                      = NaP
        0010090c 29 f5 46 f9     ldr        x9,[x9, #0xde8]=>buffer0                         = NaP
        00100910 a9 83 1d f8     stur       x9,[x29, #local_38]
        00100914 09 00 80 92     mov        x9,#-0x1
        00100918 e9 0f 00 f9     str        x9,[sp, #local_88]
        0010091c a9 03 1d f8     stur       x9,[x29, #local_40]
        00100920 bf c3 1c b8     stur       wzr,[x29, #local_44]
        00100924 a8 03 1c f8     stur       x8,[x29, #local_50]
        00100928 a0 83 5d f8     ldur       argc,[x29, #local_38]
        0010092c a1 c3 5c b8     ldur       argv,[x29, #local_44]
        00100930 a2 03 5c f8     ldur       x2,[x29, #local_50]
        00100934 a3 03 5d f8     ldur       x3,[x29, #local_40]
        00100938 76 00 00 94     bl         <EXTERNAL>::__memset_chk                         undefined __memset_chk()
        0010093c e9 0f 40 f9     ldr        x9,[sp, #local_88]
        00100940 e8 13 40 f9     ldr        x8,[sp, #local_80]
        00100944 ea 1b 40 f9     ldr        x10,[sp, #local_70]
        00100948 4a f9 46 f9     ldr        x10,[x10, #0xdf0]=>buffer1                       = NaP
        0010094c aa 83 1f f8     stur       x10,[x29, #local_18]
        00100950 a9 03 1f f8     stur       x9,[x29, #local_20]
        00100954 bf c3 1e b8     stur       wzr,[x29, #local_24]
        00100958 a8 03 1e f8     stur       x8,[x29, #local_30]
        0010095c a0 83 5f f8     ldur       argc,[x29, #local_18]
        00100960 a1 c3 5e b8     ldur       argv,[x29, #local_24]
        00100964 a2 03 5e f8     ldur       x2,[x29, #local_30]
        00100968 a3 03 5f f8     ldur       x3,[x29, #local_20]
        0010096c 69 00 00 94     bl         <EXTERNAL>::__memset_chk                         undefined __memset_chk()
        00100970 00 00 00 90     adrp       argc,0x100000
        00100974 00 44 1a 91     add        argc=>s_Enter_the_password:_00100691,argc,#0x691 = "Enter the password: "
        00100978 5a 00 00 94     bl         <EXTERNAL>::printf                               int printf(char * __format, ...)
        0010097c e9 17 40 f9     ldr        x9,[sp, #local_78]
        00100980 e8 1b 40 f9     ldr        x8,[sp, #local_70]
        00100984 21 f5 46 f9     ldr        x1,[x9, #0xde8]=>buffer0                       = NaP
        00100988 02 f9 46 f9     ldr        x2,[x8, #0xdf0]=>buffer1                         = NaP
        0010098c 00 00 00 90     adrp       argc,0x100000
        00100990 00 f4 1a 91     add        argc=>s_%8s_%8s_001006bd,argc,#0x6bd             = "%8s %8s"
        00100994 63 00 00 94     bl         <EXTERNAL>::scanf                                int scanf(char * __format, ...)
        00100998 ff 3f 00 b9     str        wzr,[sp, #local_64]
```


First we load the compiled binary executable:

In [1]:
import angr, claripy, sys

libname = "../build/arm64-v8a/06_angr_symbolic_dynamic_memory"
project = angr.Project(libname, main_opts={ "base_addr": 0x00100000 })
print(project.arch, hex(project.entry))

<Arch AARCH64 (LE)> 0x1007b4


We want start our execetion after invoke `scanf` function:(e.g.: 0x00100998)

In [2]:
after_scanf_address = 0x00100998 # int(input("after scanf address(hex): "), 16)

We prepare the **blank state** for symbolic execution:

In [3]:
blank_state = project.factory.blank_state(addr=after_scanf_address)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_MEMORY)
blank_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_REGISTERS)

Let's create some symbolic memory which pass as param into `scanf` function:
```
        0010097c e9 17 40 f9     ldr        x9,[sp, #local_78]         ; = 0x102000
        00100980 e8 1b 40 f9     ldr        x8,[sp, #local_70]         ; = 0x102000
        00100984 21 f5 46 f9     ldr        x1,[x9, #0xde8]=>buffer0   ; load memory at address, 0x102de8, the first result of malloc(0x9)
        00100988 02 f9 46 f9     ldr        x2,[x8, #0xdf0]=>buffer1   ; load memory at address, 0x102df0, the secend result of malloc(0x9)
        0010098c 00 00 00 90     adrp       x0,0x100000
        00100990 00 f4 1a 91     add        x0=>s_%8s_%8s_001006bd,argc,#0x6bd             = "%8s %8s"
        00100994 63 00 00 94     bl         <EXTERNAL>::scanf                                int scanf(char * __format, ...)

```


buffer0:
```
        001008d4 20 01 80 d2     mov        x0,#0x9                    ; param0 = 0x9
        001008d8 e0 13 00 f9     str        x0,[sp, #local_80]         ; store x0 to stack
        001008dc 89 00 00 94     bl         <EXTERNAL>::malloc         ; malloc(0x9)
        001008e0 e8 03 00 aa     mov        x8,x0                      ; save the result of malloc(ptr)
        001008e4 e0 13 40 f9     ldr        x0,[sp, #local_80]         ; restore x0 from stack
        001008e8 09 00 00 d0     adrp       x9,0x102000
        001008ec e9 17 00 f9     str        x9,[sp, #local_78]
        001008f0 28 f5 06 f9     str        x8,[x9, #0xde8]=>buffer0   ; save the result of malloc(ptr) to global data(buffer0, 0x102de8)
```

buffer1:
```
        001008f4 83 00 00 94     bl         <EXTERNAL>::malloc         ; malloc(0x9)
        001008f8 e8 13 40 f9     ldr        x8,[sp, #local_80]
        001008fc e9 17 40 f9     ldr        x9,[sp, #local_78]
        00100900 0a 00 00 d0     adrp       x10,0x102000
        00100904 ea 1b 00 f9     str        x10,[sp, #local_70]
        00100908 40 f9 06 f9     str        x0,[x10, #0xdf0]=>buffer1  ; save the result of malloc(ptr) to global data(buffer0, 0x102df0)
```

Instead of telling the binary to write to the address of the memory allocated with malloc, we can simply fake an address to any unused block of memory and overwrite the pointer to the data.

In [4]:
password0 = claripy.BVS('password0', 8 * 8)   # the heap memory
password1 = claripy.BVS('password1', 8 * 8)   # the heap memory

# symbolic heap memory
password0_ptr = 0x44444444;
password1_ptr = 0x44444444 + 8;
blank_state.memory.store(password0_ptr, password0) 
blank_state.memory.store(password1_ptr, password1) 

# buffer pointer to symbolic heap memory
buffer0_addr = 0x102de8
buffer1_addr = 0x102df0
blank_state.memory.store(buffer0_addr, password0_ptr, size=8, endness=blank_state.arch.memory_endness)
blank_state.memory.store(buffer1_addr, password1_ptr, size=8, endness=blank_state.arch.memory_endness)

Everything is ready, next we start the symbolic execution to find the path:

In [5]:
def is_successful(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Good Job.".encode() in stdout_output 

def should_abort(state):
    stdout_output = state.posix.dumps(1) # stdout
    return "Try again.".encode() in stdout_output

simulation = project.factory.simulation_manager(blank_state)
simulation.explore(find=is_successful, avoid=should_abort)
if simulation.found:
    solution0 = simulation.found[0].solver.eval(password0, cast_to=bytes).decode()
    solution1 = simulation.found[0].solver.eval(password1, cast_to=bytes).decode()

    solution = ' '.join([ solution0, solution1 ]) 
    print("Solution(password):", solution)
else:
    print("Cound not find the solution", simulation)
    list(map(lambda s: s.callstack.dbg_repr(), simulation.errored))
    list(map(lambda s: s.callstack.dbg_repr(), simulation.deadended))

Solution(password): PCBIDAMW AOADHMQD
